In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import random as r
import seaborn as sns
import matplotlib.pyplot as plt

from time import time
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GridSearchCV

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, classification_report

In [2]:
class CFG(object):
    def __init__(self,
         seed: int = 42,
         n_splits: int = 5,
         show_info: bool = False,
         ):

        self.seed = seed
        self.n_splits = n_splits
        self.show_info = show_info
        self.tr_data = "../input/criticalitypredictioninsurance/train.csv"
        self.ts_data = "../input/criticalitypredictioninsurance/test.csv"
        self.ss_data = "../input/criticalitypredictioninsurance/sample_submission.csv"
        self.model_save_path = "models"
        if not os.path.exists(self.model_save_path): os.makedirs(self.model_save_path)

cfg = CFG(seed=42, show_info=True)

In [3]:
def breaker(num: int=50, char: str="*") -> None: print("\n" + num*char + "\n")

In [4]:
class Model(object):
    def __init__(self, model_name: str, preprocessor, seed: int):
        self.model_name = model_name

        if self.model_name == "lgr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", LogisticRegression(random_state=seed, max_iter=300)),
                ]
            )
        
        elif self.model_name == "knc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", KNeighborsClassifier()),
                ]
            )
        
        elif self.model_name == "dtc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", DecisionTreeClassifier(random_state=seed)),
                ]
            )

        elif self.model_name == "etc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", ExtraTreeClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "rfc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", RandomForestClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "gbc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", GradientBoostingClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "abc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", AdaBoostClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "etcs":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", ExtraTreesClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "gnb":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", GaussianNB()),
                ]
            )
        
        elif self.model_name == "xgbc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", XGBClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "lgbmc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", LGBMClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "svc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", SVC()),
                ]
            )

In [5]:
names = ["lgr", "knc", "dtc", "etc", "rfc", "gbc", "abc", "etcs", "gnb", "xgbc", "lgbmc", "svc"]

tr_df = pd.read_csv(cfg.tr_data)
tr_df = tr_df.drop(columns=["Info_prod_2"])
X = tr_df.iloc[:, 1:-1].copy().values
y = tr_df.iloc[:, -1].copy().values
y = y - 1

if cfg.show_info:
    breaker()
    for val in set(y):
        print(f"Class {val} count : {y[y == val].shape[0]}")

features = [i for i in range(X.shape[1])]

feature_transformer = Pipeline(
    steps=[
        ("Simple_Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
        ("Standard_Scaler", StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("features", feature_transformer, features),
    ]
)

best_acc: float = 0.0
for name in names:
    fold = 1
    if cfg.show_info: breaker()
    for tr_idx, va_idx in KFold(n_splits=cfg.n_splits, random_state=cfg.seed, shuffle=True).split(X):
        X_train, X_valid, y_train, y_valid = X[tr_idx], X[va_idx], y[tr_idx], y[va_idx]
        my_pipeline = Model(name, preprocessor, cfg.seed)
        my_pipeline.model.fit(X_train, y_train)

        y_pred = my_pipeline.model.predict(X_valid)
        if cfg.show_info:
            acc = accuracy_score(y_valid, y_pred)
            print(f"{my_pipeline.model_name}, {fold}, {acc:.5f}\n")
        else:
            acc = accuracy_score(y_valid, y_pred)


        if acc > best_acc:
            best_acc = acc
            model_fold_name = f"{name}_{fold}"

            with open(os.path.join(cfg.model_save_path, f"best_model.pkl"), "wb") as fp: pickle.dump(my_pipeline.model, fp)
        
        with open(os.path.join(cfg.model_save_path, f"{name}_{fold}.pkl"), "wb") as fp: pickle.dump(my_pipeline.model, fp)
        
        fold += 1
    

if cfg.show_info: 
    breaker()
    print(f"Best Model : {model_fold_name.split('_')[0]}, Best Fold : {model_fold_name.split('_')[1]}")

breaker()


**************************************************

Class 0 count : 4360
Class 1 count : 4590
Class 2 count : 712
Class 3 count : 987
Class 4 count : 3831
Class 5 count : 7961

**************************************************

lgr, 1, 0.45712

lgr, 2, 0.45321

lgr, 3, 0.46101

lgr, 4, 0.46791

lgr, 5, 0.46190


**************************************************

knc, 1, 0.34908

knc, 2, 0.35027

knc, 3, 0.35584

knc, 4, 0.34559

knc, 5, 0.36676


**************************************************

dtc, 1, 0.42949

dtc, 2, 0.41266

dtc, 3, 0.42179

dtc, 4, 0.42981

dtc, 5, 0.41310


**************************************************

etc, 1, 0.34974

etc, 2, 0.36453

etc, 3, 0.33801

etc, 4, 0.35695

etc, 5, 0.35717


**************************************************

rfc, 1, 0.53976

rfc, 2, 0.54456

rfc, 3, 0.53119

rfc, 4, 0.54523

rfc, 5, 0.52741


**************************************************

gbc, 1, 0.54065

gbc, 2, 0.54746

gbc, 3, 0.53053

gbc, 4, 0.54835

gbc, 5, 0.54

In [6]:
ts_df = pd.read_csv(cfg.ts_data)
ts_df = ts_df.drop(columns=["Info_prod_2"])
X_test = ts_df.iloc[:, 1:].copy().values

model = pickle.load(open(os.path.join(cfg.model_save_path, f"best_model.pkl"), "rb"))
y_pred = model.predict(X_test)
y_pred = y_pred + 1

ss_df = pd.read_csv(cfg.ss_data)
ss_df.Label = y_pred.astype("int32")
ss_df.to_csv("submission.csv", index=False)